# Екатерина Кострыкина БКЛ181
#### Homework 2

In [ ]:
#! pip install spacy
#! python -m spacy download en_core_web_sm
#! pip install natasha
#! pip install flair

Imports

In [63]:
import re
import numpy as np
import pandas as pd

from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, Doc, MorphVocab

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize

import spacy
nlp = spacy.load("en_core_web_sm")

from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load('pos')

from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
2020-10-18 13:24:58,018 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt


В данных текстах встречаются слова - грамматические омонимы, то есть слова, которые пишутся одинакого, но имеют разные значения и **части речи**

В тексте на *русском языке*:
* стекло - существительное, глагол
* мой - местоимение, глагол
* три - числительное, глагол
* косой - прилагательное, существительное
* печь - глагол, существительное
* мороженное - существительное, прилагательное
* простой - существительное, глагол

В тексте на *английском языке*:
* water - поливать, вода
* like - нравится, как
* flat - квартира, плоский
* sound - здоровый, звук

In [29]:
rus_text = """в новый дом купили новое большое стекло. мой любимый сериал закончился и мне очень грустно. вася разлил 
краску на пол и мама сказала вот теперь сиди и три. летом длинную траву убирают косой. я люблю есть мороженное каждый 
летний день. маша не любит печь пироги и торты. мама попросила достать из холодильника мороженное мясо чтобы сварить 
суп. косой заяц любит прыгать по зеленой траве и собирать вкусные грибы. дайте мне пожалуйста три банки варенья с клубникой 
и два с малиной. мне не нравится простой чай мне нравится чай с лимоном и сахаром. мама сказала сначала мой посуду потом иди гулять."""

eng_text = """i do many sport activities so i consider myself as a sound and healthy person. i really love the sound of rain 
and of the surf. i like your new haircut it really suits you. my mom told me to water flowers every week at the morning. i 
bought a new pink car just like yours. if you want to be a healthy person you should drink at least two liters of water every 
day. your new flat is big and awesome i am happy you bought it. your humor is so flat i hate you and all your stupid jokes."""

In [30]:
rus_tag = ['PREP', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'ADJ', 'NOUN', 'PRON', 'ADJ', 'NOUN', 'VERB', 'CONJ', 'PRON', 'A', 'A', 
           'NOUN', 'VERB', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'NOUN', 'VERB', 'PART', 'A', 'VERB', 'CONJ', 'VERB', 'NOUN', 
           'ADJ', 'NOUN', 'VERB', 'NOUN', 'PRON', 'VERB', 'VERB', 'NOUN', 'PRON', 'ADJ', 'NOUN', 'NOUN', 'PART', 'VERB', 
           'VERB', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'VERB', 'VERB', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'CONJ', 'VERB', 'NOUN', 
           'ADJ', 'NOUN', 'VERB', 'VERB', 'PREP', 'ADJ', 'NOUN', 'CONJ', 'VERB', 'ADJ', 'NOUN', 'VERB', 'PRON', 'PART', 
           'NUM', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'NUM', 'PREP', 'NOUN', 'PRON', 'PART', 'VERB', 'ADJ', 'NOUN', 
           'PRON', 'VERB', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'VERB', 'A', 'VERB', 'NOUN', 'A', 'VERB', 'VERB']

eng_tag = ['PRON', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'PRON', 'VERB', 'PREP', 'PREP', 'DT', 'ADJ', 'CONJ', 'ADJ', 'NOUN', 
           'PRON', 'A', 'VERB', 'DT', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'DT', 'PREP', 'NOUN', 'PRON', 'VERB', 'PRON', 'ADJ', 
           'NOUN', 'PRON', 'A', 'VERB', 'PRON', 'PREP', 'NOUN', 'VERB', 'PRON', 'PREP', 'VERB', 'NOUN', 'DT', 'NOUN', 'PREP', 
           'DT', 'NOUN', 'PRON', 'VERB', 'DT', 'ADJ', 'ADJ', 'NOUN', 'A', 'PREP', 'PRON', 'PREP', 'PRON', 'VERB', 'PREP', 'VERB', 
           'DT', 'ADJ', 'NOUN', 'PRON', 'VERB', 'VERB', 'A', 'A', 'NUM', 'NOUN', 'PREP', 'NOUN', 'DT', 'NOUN', 'PRON', 'ADJ', 
           'NOUN', 'VERB', 'ADJ', 'CONJ', 'ADJ', 'PRON', 'VERB', 'ADJ', 'PRON', 'VERB', 'PRON', 'PRON', 'NOUN', 'VERB', 'A', 'ADJ', 
           'PRON', 'VERB', 'PRON', 'CONJ', 'DT', 'PRON', 'ADJ', 'NOUN']

Словарь соответсвий тегов 

In [57]:
pos_dict = {
    'S': 'NOUN','NN': 'NOUN', 'NOUN': 'NOUN', 'NNS': 'NOUN', # NOUN
    'V': 'VERB', 'GRND': 'VERB', 'VB': 'VERB', 'VERB': 'VERB', # VERB
    'VBG': 'VERB', 'VBD': 'VERB', 'VBZ': 'VERB', 'INFN': 'VERB', 'VBP': 'VERB',
    'VBN': 'VERB', 'PRTS': 'VERB', 'MD': 'VERB', 'AUX': 'VERB',
    'ADJ': 'ADJ', 'ADJF': 'ADJ', 'A': 'ADJ', 'JJ': 'ADJ', 'JJS': 'ADJ', # ADJ
    'ADV': 'A', 'ADVB': 'A', 'RB': 'A', 'WRB': 'A','EX': 'A', 'RBS': 'A', # A
    'PRON': 'PRON', 'WDT': 'PRON', 'NPRO': 'PRON', 'SPRO': 'PRON', 'APRO': 'PRON', 'ADVPRO': 'PRON', # PRON
    'PR': 'PREP', 'PREP': 'PREP', 'ADP': 'PREP', 'PRP$': 'PREP', # PREP
    'PRP': 'PREP', 'TO': 'PREP', 'IN': 'PREP', 'DET': 'DT', 'DT': 'DT', # DT
    'CC': 'CONJ', 'CONJ': 'CONJ', 'CCONJ': 'CONJ', 'SCONJ': 'CONJ', # CONJ
    'NUMR': 'NUM', 'NUM': 'NUM', 'CD': 'NUM', 'ANUM': 'NUM', 'PRCL': 'PART', 'PART': 'PART' # NUM PART
}

Функция для приведения тегов к единому формату

In [32]:
def convert(pos_list, pos_dict):
    conv_pos = []
    for i in pos_list:
        conv_pos.append(pos_dict[i])
    return conv_pos

POS-tagging

In [59]:
# for Russian

def get_Mystem_pos(text):
    pos = []
    text = word_tokenize(text)
    for word in text:
        ana = Mystem().analyze(word)
        tag = ana[0]['analysis'][0]['gr'].split(',')[0]
        if '=' in tag:
            tag = tag.split('=')[0]
            pos.append(tag)
    return pos


def get_pymorphy_pos(text):
    pos = []
    text = word_tokenize(text)
    for word in text:
        if word != '.':
            pos.append(MorphAnalyzer().parse(word)[0].tag.POS)
    return pos


def get_natasha_pos(text):
    doc = Doc(text)
    doc.segment(Segmenter())
    doc.tag_morph(NewsMorphTagger(NewsEmbedding()))
    pos = []
    for i in range(len(doc.tokens)):
        if doc.tokens[i].pos != 'PUNCT':
        pos.append(doc.tokens[i].pos)
    return pos


# for English

def get_nltk_pos(text):
    words = word_tokenize(eng_text)
    t = nltk.pos_tag(words)
    pos = []
    for i in range(len(t)):
        if t[i][1] != '.':
        pos.append(t[i][1])
    return pos


def get_spacy_pos(text):
    doc = nlp(text)
    pos = []
    for s in doc.sents:
        for t in s:
            if t.pos_ != 'PUNCT' and t.pos_ != 'SPACE':
                pos.append(t.pos_)  
    return pos


def get_flair_pos(text):
    sentence = Sentence(text)
    tagger = SequenceTagger.load('pos')
    tagger.predict(sentence)
    pos = []
    for i in sentence.to_tagged_string().split():
        if '<' in i and i != '<.>' and i != '<,>':
            pos.append(i[1:-1])
    return pos

Mystem

In [23]:
mystem_pos = get_Mystem_pos(rus_text)
conv_mystem = convert(mystem_pos, pos_dict)
accuracy_score(rus_tag, conv_mystem)

0.89

Pymorpy

In [24]:
pymorphy_pos = get_pymorphy_pos(rus_text)
conv_pymorphy = convert(pymorphy_pos, pos_dict)
accuracy_score(rus_tag, conv_pymorphy)

0.93

NATASHA

In [25]:
natasha_pos = get_natasha_pos(rus_text)
conv_natasha = convert(natasha_pos, pos_dict)
accuracy_score(rus_tag, conv_natasha)

0.94

Spacy


In [35]:
spacy_pos = get_spacy_pos(eng_text)
conv_spacy = convert(spacy_pos, pos_dict)
accuracy_score(eng_tag, conv_spacy)

0.8514851485148515

Flair

In [36]:
flair_pos = get_flair_pos(eng_text)
conv_flair = convert(flair_pos, pos_dict)
accuracy_score(eng_tag, conv_flair)

2020-10-18 12:35:44,520 loading file /root/.flair/models/en-pos-ontonotes-v0.5.pt


0.7722772277227723

NLTK

In [58]:
nltk_pos = get_nltk_pos(eng_text)
conv_nltk = convert(nltk_pos, pos_dict)
accuracy_score(eng_tag, conv_nltk)

0.7029702970297029

### Функция поиска групп биграмм:
1. "не" + прилагательное 
    - например, так как в хорошем отзыве могут встретиться "не" и "плохой", которые по отдельности могут относится к группе "негативных" слов, а объединив их в одно "не плохой" мы получим "хороший"
    
    
2. прилагательное + существительное 
3. наречие + глагол
    - так как прилагательные и наречия являются оценочными средствами, было бы полезно выделить к чему именно они относятся


Так как наилучший результат был достигнут при использовании Natasha, здесь будем использовать этот парсер 

In [85]:
def get_bigrams(text):
    doc = Doc(text)
    doc.segment(Segmenter())
    doc.tag_morph(NewsMorphTagger(NewsEmbedding()))
    for i in doc.tokens:
        i.lemmatize(MorphVocab())

    lemma_tag = [[i.lemma, i.pos] for i in doc.tokens]

    bigrams = []
    for i in range(len(lemma_tag)-1):
        lemma = lemma_tag[i][0]
        next_lemma = lemma_tag[i+1][0]
        tag = lemma_tag[i][1]
        next_tag = lemma_tag[i+1][1]
        if lemma == 'не' and next_tag == 'ADJ':
            bigrams.append(' '.join([lemma, next_lemma]))
        elif tag == 'A' and next_tag == 'VERB':
            bigrams.append(' '.join([lemma, next_lemma]))
        elif tag == 'ADJ' and next_tag == 'NOUN':
            bigrams.append(' '.join([lemma, next_lemma]))

    return bigrams

In [79]:
get_bigrams(rus_text)

['новый дом',
 'большой стекло',
 'любимый сериал',
 'длинный трава',
 'летний день',
 'мороженный мясо',
 'косой заяц',
 'зеленый трава',
 'вкусный гриб',
 'простой чай']